In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [98]:
pd. __version__

'2.2.3'

In [82]:
df_orig = pd.read_csv("cbecs2018_final_public.csv")
df_orig.head()

,PUBID,REGION,CENDIV,PBA,PUBCLIM,SQFT,SQFTC,WLCNS,RFCNS,RFCOOL,...,ZMFBTU,ZMFEXP,ZELCNS,ZELEXP,ZNGCNS,ZNGEXP,ZFKCNS,ZFKEXP,ZDHBTU,ZDHEXP
0,1,3,5,2,3,210000,8,1,4,2,...,0,0,0,0,9,9,1,1,0,0
1,2,4,9,2,4,28000,5,1,6,1,...,0,0,0,0,0,0,9,9,9,9
2,3,3,5,8,4,2100,2,1,4,2,...,0,0,0,0,9,9,9,9,9,9
3,4,3,7,5,5,240000,8,2,6,1,...,0,0,0,0,1,1,9,9,9,9
4,5,1,2,5,3,295000,8,3,6,2,...,0,0,0,0,0,0,9,9,9,9


In [83]:
display(df_orig.loc[(df_orig['PUBID'] > 0) & (df_orig['PBA'] == 2) & (df_orig['PUBCLIM'] == 4) & (df_orig['SQFT'] > 0), ['PUBID', 'PBA']])

,PUBID,PBA
1,2,2
18,19,2
21,22,2
29,30,2
31,32,2
...,...,...
6377,6378,2
6399,6400,2
6406,6407,2
6429,6430,2


In [84]:
df_wk = df_orig.copy()
df_wk.head(2)

,PUBID,REGION,CENDIV,PBA,PUBCLIM,SQFT,SQFTC,WLCNS,RFCNS,RFCOOL,...,ZMFBTU,ZMFEXP,ZELCNS,ZELEXP,ZNGCNS,ZNGEXP,ZFKCNS,ZFKEXP,ZDHBTU,ZDHEXP
0,1,3,5,2,3,210000,8,1,4,2,...,0,0,0,0,9,9,1,1,0,0
1,2,4,9,2,4,28000,5,1,6,1,...,0,0,0,0,0,0,9,9,9,9


#### creates df_wk dataset

In [85]:
df_wk = df_orig.iloc[:,[0,3,4,5,7,8,11,12,13,18,21,77,103,104,113,114,133,
    134,139,140,141,295,378,379,425,561,562,566,569,567,568,570,571]]
df_wk.head()

,PUBID,PBA,PUBCLIM,SQFT,WLCNS,RFCNS,BLDSHP,GLSSPC,NFLOOR,NELVTR,...,NGWATR,NGOTH,HDD65,CDD65,ELBTU,NGBTU,ELCNS,ELEXP,NGCNS,NGEXP
0,1,2,3,210000,1,4,6,3,994,8.0,...,2,2,4463,1759,18708970.0,NaN,5483285.0,775800.0,NaN,NaN
1,2,2,4,28000,1,6,11,3,5,1.0,...,1,2,2424,189,1528667.0,201988.0,448027.0,77933.0,1946.0,4097.0
2,3,8,4,2100,1,4,2,3,1,NaN,...,2,2,3218,2403,52387.0,NaN,15354.0,3472.0,NaN,NaN
3,4,5,5,240000,2,6,2,2,1,NaN,...,1,2,1045,3041,1974255.0,1211520.0,578621.0,41197.0,11672.0,9862.0
4,5,5,3,295000,3,6,1,1,1,NaN,...,2,2,5076,1316,1290564.0,3386445.0,378243.0,33688.0,32625.0,35297.0


#### commit

#### Add cold degree days (CDD) and hot degree days (HDD) low to high ranges and averages to df_wk.
- The code cell below creates the df_clim dataframe.
- The data is from "Climate Data for Building Design Standards".

In [86]:
data = {'PUBCLIM': [2, 3, 4],
        'CDD_LOW': [1800, 2700, 4500],
        'CDD_HI': [6300, 6300, 6300],
        'HDD_LOW': [5400, 3600, 1800],
        'HDD_HI': [7200, 5400, 3600],
        'CDD_AVG': [4050, 4500, 5400],
        'HDD_AVG': [6300, 4500, 2700]
        }

df_clim = pd.DataFrame(data)
print(df_clim)

   PUBCLIM  CDD_LOW  CDD_HI  HDD_LOW  HDD_HI  CDD_AVG  HDD_AVG
0        2     1800    6300     5400    7200     4050     6300
1        3     2700    6300     3600    5400     4500     4500
2        4     4500    6300     1800    3600     5400     2700


- This code cell joins the df_clim to df_wk using the common PUBCLIM column to align the rows based on the 2, 3, and 4 climate zones.

In [87]:
df_wk = pd.merge(df_wk, df_clim, on='PUBCLIM', how='left')
print(df_wk)

      PUBID  PBA  PUBCLIM    SQFT  WLCNS  RFCNS  BLDSHP  GLSSPC  NFLOOR  \
0         1    2        3  210000      1      4       6       3     994   
1         2    2        4   28000      1      6      11       3       5   
2         3    8        4    2100      1      4       2       3       1   
3         4    5        5  240000      2      6       2       2       1   
4         5    5        3  295000      3      6       1       1       1   
...     ...  ...      ...     ...    ...    ...     ...     ...     ...   
6431   6432   14        2  130000      1      1       2       3       3   
6432   6433    1        4    1050      1      5       1       3       1   
6433   6434    2        4  122000      2      6      11       5       5   
6434   6435   23        4   15000      3      5       2       4       1   
6435   6436   14        7  188000      1      6       4       2       7   

      NELVTR  ...      ELCNS     ELEXP    NGCNS    NGEXP  CDD_LOW  CDD_HI  \
0        8.0  ...  548

#### end commit

#### commit

#### Filtered df_wk where the principle business activity (PBA) is 2 = office building, and the buildings are in climate zones PUBCLIM = 2 or 3 or 4.

In [88]:
df_wk_filtered = df_wk[(df_wk['PBA'] == 2) & (df_wk['PUBCLIM'].isin([2, 3, 4]))]
print(df_wk_filtered.shape)
print(df_wk_filtered)

(1018, 39)
      PUBID  PBA  PUBCLIM    SQFT  WLCNS  RFCNS  BLDSHP  GLSSPC  NFLOOR  \
0         1    2        3  210000      1      4       6       3     994   
1         2    2        4   28000      1      6      11       3       5   
12       13    2        3   30500      1      6       2       5       3   
18       19    2        4  184000      1      6       3       3       3   
20       21    2        3  240000      6      5       2       6       4   
...     ...  ...      ...     ...    ...    ...     ...     ...     ...   
6420   6421    2        3  430000      1      7       2       4     995   
6421   6422    2        2  385000      1      2       5       5       4   
6427   6428    2        2  265000      2      8       1       5     995   
6429   6430    2        4   54000      1      6       2       3       3   
6433   6434    2        4  122000      2      6      11       5       5   

      NELVTR  ...      ELCNS     ELEXP     NGCNS    NGEXP  CDD_LOW  CDD_HI  \
0        8

- The row index was reset so the new index is sequencial.

In [89]:
# resetting the df_wk row index
df_wk_filtered = df_wk_filtered.reset_index()
print(df_wk_filtered.shape)
print(df_wk_filtered)

(1018, 40)
      index  PUBID  PBA  PUBCLIM    SQFT  WLCNS  RFCNS  BLDSHP  GLSSPC  \
0         0      1    2        3  210000      1      4       6       3   
1         1      2    2        4   28000      1      6      11       3   
2        12     13    2        3   30500      1      6       2       5   
3        18     19    2        4  184000      1      6       3       3   
4        20     21    2        3  240000      6      5       2       6   
...     ...    ...  ...      ...     ...    ...    ...     ...     ...   
1013   6420   6421    2        3  430000      1      7       2       4   
1014   6421   6422    2        2  385000      1      2       5       5   
1015   6427   6428    2        2  265000      2      8       1       5   
1016   6429   6430    2        4   54000      1      6       2       3   
1017   6433   6434    2        4  122000      2      6      11       5   

      NFLOOR  ...      ELCNS     ELEXP     NGCNS    NGEXP  CDD_LOW  CDD_HI  \
0        994  ...  548

#### end commit

#### commit

#### Data summary.

In [109]:
def run_sumarize(df):
    print('The number of rows and columns is:')
    print(df.shape)
    print('\nListing of all columns and information about them is:')
    print(df.info())
    print('\nThe number of rows w/ null values in each column is:')
    print(df.isnull().sum())
    print('\nThe basic statistic and descriptive values for the selected columns are: ')
    print(df[['SQFT', 'ELCNS', 'NGCNS']].describe().round(2))
    
run_sumarize(df_wk_filtered)

The number of rows and columns is:
(1018, 40)

Listing of all columns and information about them is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 40 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      1018 non-null   int64  
 1   PUBID      1018 non-null   int64  
 2   PBA        1018 non-null   int64  
 3   PUBCLIM    1018 non-null   int64  
 4   SQFT       1018 non-null   int64  
 5   WLCNS      1018 non-null   int64  
 6   RFCNS      1018 non-null   int64  
 7   BLDSHP     1018 non-null   int64  
 8   GLSSPC     1018 non-null   int64  
 9   NFLOOR     1018 non-null   int64  
 10  NELVTR     1018 non-null   float64
 11  YRCONC     1018 non-null   int64  
 12  WKHRSC     1018 non-null   int64  
 13  ELHT1      1018 non-null   int64  
 14  NGHT1      1018 non-null   int64  
 15  ELHT2      1018 non-null   int64  
 16  NGHT2      1018 non-null   int64  
 17  BOILER_EL  583 non-null    

In [ ]:
def run_value_counts(df):
    print('The number of responses for each climate zone catagory is:')
    print(df['PUBCLIM'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for each building shape catagory is:')
    print(df['BLDSHP'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for each exterior glass percent catagory is:')
    print(df['GLSSPC'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for number of elevators is:')
    print(df['NELVTR'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for each construction year catagory is:')
    print(df['YRCONC'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for each weekly building hours open catagory is:')
    print(df['WKHRSC'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for main heating by electricity is:')
    print(df['ELHT1'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for main heating by natural gas is:')
    print(df['NGHT1'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for air conditioning by electricity is:')
    print(df['ELCOOL'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for water heating by electricity is:')
    print(df['ELWATR'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for water heating by electricity is:')
    print(df['NGWATR'].value_counts().sort_values(ascending=False))
    print('\nThe number of responses for number of floors catagory is:')
    print(df['NFLOOR'].value_counts().sort_values(ascending=False))
run_value_counts(df_wk_filtered)

The number of responses for each climate zone catagory is:
PUBCLIM
3    410
4    326
2    282
Name: count, dtype: int64

The number of responses for each building shape catagory is:
BLDSHP
2     588
1     141
11     71
9      64
3      47
4      35
6      31
5      19
8      13
7       6
10      3
Name: count, dtype: int64

The number of responses for each exterior glass percent catagory is:
GLSSPC
3    327
4    247
2    181
5    145
6     69
1     49
Name: count, dtype: int64

The number of responses for number of elevators = 1 - 30, and > 30 catagories is:
NELVTR
1.0      461
2.0      117
3.0       96
4.0       61
6.0       61
5.0       50
7.0       31
8.0       22
10.0      22
9.0       20
12.0      17
11.0      14
14.0      12
13.0       8
17.0       5
15.0       5
16.0       4
18.0       3
22.0       2
19.0       2
20.0       2
28.0       1
25.0       1
995.0      1
Name: count, dtype: int64

The number of responses for each construction year catagory is:
YRCONC
6    235
8    167


#### Data clean up.

##### The formula below calculates a fill-in value for the rows with missing data in HGCNS column.  See describe() output at the end of the run_sumarize function.  
- The mean of the existing values is much higher than the 50% percentile (median) value.
- This means the distribution of the values is skewed higher than the median value.
- I chose a value between the mean and median due to the mean being so much higher than the median value. 

In [ ]:
# fillna value for NGCNS missing values
mean = 29085
median = 11565
result = ((mean - median) / 2) + median
print(result)

- NGCNS (annual natural gas used) and NELVTR (number of elevators) fillna values to replace missing values.
- The fillna value for the NELVTR missing data cells was the most used value.

In [ ]:
# fillna value into the missing value cells
# a Pandas FutureWarning message will appear as output 
df_wk_filtered['NGCNS'].fillna(20325, inplace=True)

In [ ]:
# fillna value for NELVTR
# a Pandas FutureWarning message will appear as output 
df_wk_filtered['NELVTR'].fillna(1, inplace=True)

- Verify the fillna values were successfully replaced.

In [ ]:
# verify missing values have been replaced.
print('The number of rows and columns is:')
print(df_wk_filtered.shape)
print('\nThe number of rows w/ null values in each column is:')
print(df_wk_filtered[['NGCNS','NELVTR']].isnull().sum())

#### end commit

#### create calc'd columns

# funtion to create the calculate the CDD_RANGE
def calc_cdd_range(row):
  """Calculates the CDD_range value for a row.
  Args:
    row: A row of the DataFrame.
  Returns:
    'yes' if CDD_LOW <= CDD65 <= CDD_HI, otherwise 'no'.
  """
  if row['CDD_LOW'] <= row['CDD65'] <= row['CDD_HI']:
    return 'yes'
  else:
    return 'no'

# function is applied to df_wk_clean to create the new column
df_wk_filtered['CDD_RANGE'] = df_wk_filtered.apply(calculate_cdd_range, axis=1)

In [107]:
column = df_wk_filtered['NELVTR']
print(column)

0       8.0
1       1.0
2       1.0
3       3.0
4       4.0
       ... 
1013    7.0
1014    5.0
1015    6.0
1016    2.0
1017    2.0
Name: NELVTR, Length: 1018, dtype: float64
